In [11]:
allOS = [] # list of all OS

In [12]:
from cvzone.HandTrackingModule import HandDetector # pip install cvzone
import cv2 
import boto3 # pip install boto3
def add_OS(): # add an instance
    ec2 = boto3.resource("ec2") # create an ec2 (Elastic Compute Cloud) resource
    instances = ec2.create_instances(   # create an instance
        ImageId="ami-0a2acf24c0d86e927", # Amazon Machine Image (AMI) ID that will be used to launch the instance
        MinCount=1, # minimum number of instances to launch
        MaxCount=1, # maximum number of instances to launch
        InstanceType="t2.micro", # free tier, low RAM and CPU
        SecurityGroupIds=["sg-061fc03509c386fb6"] # used for defining rules of inbound and outbound traffic
    )
    allOS.append(instances[0].id) # add the instance id to the array
    print(len(allOS))
    
def deleteOS(): # delete an instance
    ec2 = boto3.resource("ec2") # create an ec2 resource
    if(len(allOS)>0):
        lastOS = allOS.pop()
        ec2.instances.filter(InstanceIds = [lastOS]).terminate() # terminate the instance
        print(len(allOS))
    else:
        print("array empty")
    
cap = cv2.VideoCapture(0) # capture video from default webcam
while(True):
    status, photo = cap.read() # read the video frame by frame, stauts is a boolean representing if the frame was read correctly
#     crop = photo[100:400,100:400] # might need to crop the image
    cv2.imshow("my-image",photo) # show the image
    if (cv2.waitKey(1000) == 13): # if enter is pressed, exit the loop, release the video capture, and destroy all windows (every 1 second)
        cap.release()
        cv2.destroyAllWindows()
        break;
    detector = HandDetector(maxHands=1) # create a hand detector object
    ishand = detector.findHands(photo,draw=False) # find hands in the image, draw true will draw the hand landmarks
    if(ishand): 
        fingerlist = ishand[0]
        fingers = detector.fingersUp(fingerlist)
        print(fingers)
        print(ishand[0]["lmList"])
        x = ishand[0]["lmList"][4][0] - ishand[0]["lmList"][8][0]; # dist b/w tip of thumb
        y = ishand[0]["lmList"][4][1] - ishand[0]["lmList"][8][1]; # dist b/w tip of index finger
        if(abs(x)<=20 and abs(y)<=20): # if index and thumb are touching on tip, add an instance
            print("Touching")
            cv2.waitKey(3000)
            add_OS()
            cap.release()
            cv2.destroyAllWindows()
            break;
        elif(fingers == [0,1,1,0,0]): # if index and middle finger are shown, delete an instance
            print("2 fingers")
            cv2.waitKey(3000)
            deleteOS()
            cap.release()
            cv2.destroyAllWindows()
            break;
        else:
            print("not touching")
    else:
        print("no hands")

[]
no hands
[]
no hands
[]
no hands
[]
no hands
[]
no hands
[]
no hands
[{'lmList': [[219, 456, 0], [337, 420, -67], [441, 335, -102], [523, 264, -131], [611, 226, -161], [356, 173, -40], [393, 70, -78], [420, 3, -115], [443, -48, -142], [287, 156, -36], [307, 41, -68], [324, -34, -107], [340, -87, -135], [215, 165, -41], [227, 49, -74], [237, -23, -107], [248, -80, -130], [139, 197, -52], [133, 98, -83], [133, 32, -102], [136, -25, -115]], 'bbox': (133, -87, 478, 543), 'center': (372, 184), 'type': 'Right'}]
[1, 1, 1, 1, 1]
[[219, 456, 0], [337, 420, -67], [441, 335, -102], [523, 264, -131], [611, 226, -161], [356, 173, -40], [393, 70, -78], [420, 3, -115], [443, -48, -142], [287, 156, -36], [307, 41, -68], [324, -34, -107], [340, -87, -135], [215, 165, -41], [227, 49, -74], [237, -23, -107], [248, -80, -130], [139, 197, -52], [133, 98, -83], [133, 32, -102], [136, -25, -115]]
not touching
[{'lmList': [[289, 382, 0], [338, 368, -20], [379, 334, -28], [410, 302, -35], [444, 292, -41], 